In [1]:
from utils import UNet, SegmentationDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

dataset = SegmentationDataset("SA1B_Meta_AI_Segmentation_Dataset/")
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
model = UNet(in_channels=3, out_channels=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [2]:
def train_unet(model, dataset, epochs=10, lr=1e-4):
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # For binary segmentation

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for imgs, masks in dataloader:
            imgs, masks = imgs.cuda(), masks.cuda()  # Move to GPU if available

            # Forward pass
            preds = model(imgs)
            loss = criterion(preds, masks.unsqueeze(1))  # Add channel dim to masks
            epoch_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

    return model

In [ ]:
model.train()
for epoch in range(2):
	epoch_loss = 0
	for idx, (image, mask) in enumerate(dataloader):
		outputs = model(image)
		loss = criterion(outputs, mask)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		# outputs.weights
		print(f'epoch: {epoch+1} | loss: {epoch_loss}')


In [ ]:
print(dataset)